<a href="https://colab.research.google.com/github/gyuchangShim/MJU_AI/blob/VGG16-CNN/Colaboratory%EC%97%90_%EC%98%A4%EC%8B%A0_%EA%B2%83%EC%9D%84_%ED%99%98%EC%98%81%ED%95%A9%EB%8B%88%EB%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib

In [ ]:
print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tf.test.is_gpu_available()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [10]:
def VGG16():
    tf.random.set_seed(2)
    model = tf.keras.models.Sequential([
        # input = 224 x 224 x 3

        # 224 x 224 x 64
        layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', input_shape=(224, 224, 3)),
        layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'),

        # 112 x 112 x 64
        layers.MaxPool2D((2, 2), padding='same'),

        # 112 x 112 x 128
        layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'),

        # 56 x 56 x 128
        layers.MaxPool2D((2, 2), padding='same'),

        # 56 x 56 x 256
        layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'),

        # 28 x 28 x 256
        layers.MaxPool2D((2, 2), padding='same'),

        # 28 x 28 x 512
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'),

        # 14 x 14 x 512
        layers.MaxPool2D((2, 2), padding='same'),

        # 14 x 14 x 512
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'),
        layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu'),

        # 7 x 7 x 512
        layers.MaxPool2D((2, 2), padding='same'),

        # 1 x 1 x 25088
        layers.Flatten(),
        layers.Dropout(0.5),

        # 1 x 1 x 4096
        layers.Dense(4096, activation='relu'),

        # 1 x 1 x 4096
        layers.Dense(4096, activation='relu'),

        # 1 x 1 x 1000
        layers.Dense(1000, activation='relu'),

        # 1 x 1 x 2
        layers.Dense(2, activation='softmax'),
    ])
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                  loss='categorical_crossentropy',
                  metrics=['acc'])

    return model

In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
import shutil
import json
from sklearn.model_selection import train_test_split

# 원본 데이터셋 경로
original_labels_path = '/content/gdrive/MyDrive/dataset/VL_damage_part/'
original_images_path = '/content/gdrive/MyDrive/dataset/VS_damage_part/'
# 나눌 데이터셋 경로
train_dataset_path = '/content/drive/MyDrive/dataset/train_set/'
valid_dataset_path = '/content/drive/MyDrive/dataset/validation_set/'

# 폴더 생성
os.makedirs(train_dataset_path, exist_ok=True)
os.makedirs(valid_dataset_path, exist_ok=True)

# 이미지와 라벨 로드
labels_list = []
for label_file in os.listdir(original_labels_path):
    with open(os.path.join(original_labels_path, label_file), 'r') as f:
        label_data = json.load(f)
        labels_list.append(label_data)

# 이미지 리스트 가져오기
images = [f for f in os.listdir(original_images_path) if os.path.isfile(os.path.join(original_images_path, f))]
print(len(images))

# 이미지 파일명에서 확장자를 제거한 딕셔너리 생성
image_files_dict = {os.path.splitext(img)[0]: img for img in images}

# 이미지와 라벨 매칭
labeled_images = []
for img_idx, img_file in enumerate(images):
    # 이미지 파일명에서 확장자 제거
    img_name = os.path.splitext(img_file)[0]
    if img_name in image_files_dict:
        label_idx = -1
        for label_data in labels_list:
            if img_name in label_data:
                label_idx = label_data.index(img_name)
                break
        if label_idx != -1:
            labeled_images.append((img_file, labels_list[label_idx]))
        else:
            print(f"No label found for image: {img_file}")
    else:
        print(f"No corresponding image found for label: {img_file}")

# 디렉토리에 이미지와 라벨 복사
for img, label in train_images:
    label_dir = os.path.join(train_dataset_path, label)
    os.makedirs(label_dir, exist_ok=True)
    src_path = os.path.join(original_images_path, img)
    dst_path = os.path.join(label_dir, img)
    shutil.copy(src_path, dst_path)

for img, label in valid_images:
    label_dir = os.path.join(valid_dataset_path, label)
    os.makedirs(label_dir, exist_ok=True)
    src_path = os.path.join(original_images_path, img)
    dst_path = os.path.join(label_dir, img)
    shutil.copy(src_path, dst_path)

In [ ]:
train_dataset_path = '/content/drive/MyDrive/dataset/train_set'
train_data_generator = ImageDataGenerator(rescale=1. / 255)
train_dataset = train_data_generator.flow_from_directory(train_dataset_path,
                                                         shuffle=True,
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='categorical')


valid_dataset_path = '/content/drive/MyDrive/dataset/validation_set'
valid_data_generator = ImageDataGenerator(rescale=1. / 255)
valid_dataset = valid_data_generator.flow_from_directory(valid_dataset_path,
                                                         shuffle=True,
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='categorical')


In [ ]:
# Train
print("Start Train!")
model = VGG16()
model.summary()
train = model.fit(train_dataset, epochs=5, validation_data=valid_dataset)

In [ ]:
# Accuracy graph
plt.figure(1)
plt.plot(train.history['acc'])
plt.plot(train.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('VGG16_Accuracy_1.png')
print("Saved Accuracy graph")

In [ ]:
# Loss graph
plt.figure(2)
plt.plot(train.history['loss'])
plt.plot(train.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('VGG16_Loss_1.png')
print("Saved Loss graph")

model.save('VGG16.h5')